In [1]:
import pandas as pd
import numpy as np
from sklearn.utils.extmath import randomized_svd

Cloning into dataset repo

In [2]:
!git clone https://github.com/gagan-iitb/DS-503

Cloning into 'DS-503'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (609/609), done.
remote: Compressing objects: 100% (558/558), done.
remote: Total 609 (delta 337), reused 66 (delta 30), pack-reused 0
Receiving objects: 100% (609/609), 84.67 MiB | 24.40 MiB/s, done.
Resolving deltas: 100% (337/337), done.


Loading data into dataframes

In [3]:
train_set = pd.read_csv('/content/DS-503/Assignments/Assignment2-CollabFiltering/Training_set.csv') # train set
val_set = pd.read_csv('/content/DS-503/Assignments/Assignment2-CollabFiltering/Validation_set.csv') # validation set
test_set = pd.read_csv('/content/DS-503/Assignments/Assignment2-CollabFiltering/Test_set.csv')  # test set
val_gt = pd.read_csv('/content/DS-503/Assignments/Assignment2-CollabFiltering/Validation_set_with_groundtruth.csv') # ground truth

Pseudo-Inverse Approach

Assume that we have to guess the rating of an item for a given user X. We know the rating of 30 items for this user (in our validation or test set). Now, we take all the users Y, who have rated all 100 items. We  estimate that the ratings of user X are a linear weighted combination of the ratings of these users Y. So, we have an under-determined system with 30 equations and Y variables. Using the pseudo-inverse approach, we can compute the least-square estimate of the weights. Using these weights, we can estimate the ratings of all the 70 items for user X as weighted combinations of the ratings of users Y.  

Finding Y from training set

In [ ]:
Y = []

for j in range(len(val_set)):
  l = train_set.loc[j, :].values.tolist()[1:]
  flag = False
  for i in range(len(l)):
    if l[i] == 99.0:
      flag = True
      break
  if not flag: Y.append(l)

# Y to matrix
Y = np.array(Y)

Input user ID and find 30 ratings for this user

In [ ]:
x = int(input('Enter user id :'))
val_l = val_set.loc[x, :].values.tolist()[1:] # ratings of user x from val set

ratings = []  # to store ratings
r_indices = []  # to store indices of rated items
missing = []

# finding rated items

for i in range(len(val_l)):

  if val_l[i] != 99.0:
    ratings.append(val_l[i])
    r_indices.append(i)

  else:
    missing.append(i)

# if we find 30 ratings from val set we are done. Else get ground truth ratings

if len(ratings) < 30:
  gt_l = val_gt.loc[x, :].values.tolist()[1:] # ratings of user x from gt
  i = 0
  while len(ratings) < 30:

    if gt_l[i] != 99.0 and i in missing:
      ratings.append(gt_l[i])
      r_indices.append(i)

    i += 1

# converting ratings to numpy array

ratings = np.array(ratings)
print('Done')

Enter user id56
Done


Using pseudo inverse to estimate all ratings of user x

In [ ]:
Y_30 = Y[:, r_indices].T
U, Sigma, VT = randomized_svd(Y_30, 
                              n_components=30,
                              random_state=None)

V = VT.T
theta = V@np.linalg.pinv(np.diag(Sigma))@U.T@ratings

print('Ratings of user {} : {}'.format(x, theta@Y.tolist()))

Ratings of user 56 : [-1.99191821 -6.12        1.10622163 -2.51515659 -1.12573241 -0.15768791
 -2.90945783 -0.04181989 -0.33413767 -0.44        1.37183564  3.35
 -2.39318002  2.23056744 -1.90577338 -4.60800139  0.68883863 -2.33199036
  0.03918475  0.52430498 -0.05        5.44       -0.52997374 -6.21
  0.64002495  0.63        0.64723401  5.15        1.15088     0.60804341
 -2.55207694  1.13222328 -2.52        3.25        1.65637952  1.48683704
 -0.55866246 -2.23054358  2.2593437   1.70812809  0.73116081  3.81746562
  1.94       -4.01180069 -0.35970364  2.41915953  1.36        0.1939729
  0.43231819  0.66555626 -0.34        1.1547122   0.83562293  9.37
  1.8        -0.59342912 -2.32869825 -6.89        3.35       -1.14398884
 -5.15        0.68674253  4.17        0.54831371  9.27       -2.14
  0.27101681 -4.03        0.30140594  0.77110129  3.23559727  2.19117805
  3.07049467  1.17076657  2.68154098  2.9650118   1.4839693   4.9
  3.25        2.67        1.12340314  3.95516825  4.42038184  

From this we can ibtain the remaining 70 missing ratings

Nearest Neighbor approach: Item-Item Collaborative Filtering

In this approach, the rating of an item is estimated using the ratings of similar items. Assume that we have to guess the rating of an item “i” for a given user X. We first find K (e.g K=3) other items that have been rated  by  X  and  are  most  similar  to  item  “i”. Similarity can be computed using the Pearson Correlation Coefficient (take dot product after subtracting the mean). Using the correlation coefficients as weights, we can estimate the rating as the weighted average of the known ratings of items by that user. We also 
adjust for the deviation of the average ratings of the item “i” and user “X” from the overall average rating of all the items in the dataset. 


In [5]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [12]:
user = int(input('Enter user id :')) # user id
item = int(input('Enter item :'))  # item rating to be guessed

l = train_set.loc[user, :].values.tolist()[1:]

if l[item] != 99.0: # if rating exists
  print('Rating exists : {}'.format(l[item]))
  raise StopExecution # this is to break cell execution



Enter user id :1
Enter item :4
Rating exists : 9.03


StopExecution: ignored